In [1]:
import pandas as pd
import numpy as np
import os

In [ ]:
# Create bill of materials dataframe
bill_of_materials_files = ['data/bill_of_materials_product_A.csv', 
                           'data/bill_of_materials_product_B (1).csv',
                           'data/bill_of_materials_product_C.csv',
                           'data/bill_of_materials_product_D_revised.csv']

bill_of_materials_df = None
products = []

# Convert each product bill of materials into an item in the dictionary
for file in bill_of_materials_files:
    if file == 'data/bill_of_materials_product_D_revised.csv':
        df = pd.read_csv(file, delimiter=';')
        df = df.rename(columns={'pcs': 'Quantity'})
        df = df[['Component', 'Price €', 'Quantity']]
        df['Component'] = df['Component'].astype(str)
        df['Price €'] = df['Price €'].str.replace(',', '.').astype(float)
        df['Quantity'] = df['Quantity'].str.replace(',', '.').astype(float).astype(int)
        product_name = file.split('_')[-2].split('.')[0][0]
    else:
        df = pd.read_csv(file)
        df['Component'] = df['Component'].astype(str)
        product_name = file.split('_')[-1].split('.')[0][0]

    products.append(product_name)

    df['product_id'] = product_name
    df = df.rename(columns={'Component': 'component_id'})
    df = df.rename(columns={'Price €': 'price'})
    df = df.rename(columns={'Quantity': 'amount'})

    # Fix component_id format
    df['component_id'] = df['component_id'].apply(lambda x: 
        # If already in correct format (A.12), keep as is
        x if len(x) == 4 and '.' in x
        # If dot is missing (A12), insert dot at position 1
        else x[0] + '.' + x[1:] if len(x) == 3
        # If number is single digit (A.1), add leading zero
        else x[0] + '.' + x[2:].zfill(2) if len(x) == 3 and '.' in x
        # For any other case, raise error in check
        else x)

    # Verify all component_ids are now in correct format
    assert df['component_id'].str.match(r'[A-Z]\.\d{2}').all(), 'Component ID format is incorrect'

    if bill_of_materials_df is None:
        bill_of_materials_df = df
    else:
        bill_of_materials_df = pd.concat([bill_of_materials_df, df])

bill_of_materials_df

In [ ]:
# Check if the bill of materials dataframe is correct
# Check if the unique products are correct
assert bill_of_materials_df['product_id'].nunique() == 4, 'The number of unique products is incorrect'

# Check that price is a float
assert bill_of_materials_df['price'].dtype == np.float64, 'Price is not a float'

# Check that amount is an integer
assert bill_of_materials_df['amount'].dtype == np.int64, 'Amount is not an integer'

# Check that the component_id is in format 'Character.Number'
assert bill_of_materials_df['component_id'].str.match(r'[A-Z]\.\d').all(), 'Component ID is not in format Character.Number'

print('Bill of materials dataframe is correct')

In [ ]:
# Product dataframe
product_df = pd.DataFrame(products, columns=['product_id'])
product_df

In [ ]:
# Component dataframe
component_df = bill_of_materials_df[['component_id']].drop_duplicates()
component_df

In [ ]:
# Product-Component dataframe
product_component_dataframe = bill_of_materials_df[['product_id', 'component_id', 'amount', 'price']]
product_component_dataframe.rename(columns={'price': 'expected_price'}, inplace=True)
product_component_dataframe

In [ ]:
# Check if the product-component dataframe is correct
# Check if the unique products are correct
assert product_component_dataframe['product_id'].nunique() == 4, 'The number of unique products is incorrect'

# Check if the unique components are correct
assert product_component_dataframe['component_id'].nunique() == 61, 'The number of unique components is incorrect'

# Check that expected price is a float
assert product_component_dataframe['expected_price'].dtype == np.float64, 'Expected price is not a float'

# Check that amount is an integer
assert product_component_dataframe['amount'].dtype == np.int64, 'Amount is not an integer'

print('Product-component dataframe is correct')

In [ ]:
# Supplier-Component dataframe
supplier_component_df = pd.read_csv('data/updated_suppliers_to_components_with_prices.csv')
supplier_component_df = supplier_component_df.rename(columns={'Component': 'component_id'})
supplier_component_df = supplier_component_df.rename(columns={'Supplier': 'supplier_id'})
supplier_component_df = supplier_component_df.rename(columns={'Price €': 'price'})
supplier_component_df

In [ ]:
# Check if the supplier-component dataframe is correct
# Check if the unique suppliers are correct
assert supplier_component_df['supplier_id'].nunique() == 20, 'The number of unique suppliers is incorrect'

# Check that price is a float
assert supplier_component_df['price'].dtype == np.float64, 'Price is not a float'

# Check that the component_id is in format 'Character.Number'
assert supplier_component_df['component_id'].str.match(r'[A-Z]\.\d').all(), 'Component ID is not in format Character.Number'

# Check that the component_id is found in the component dataframe
assert supplier_component_df['component_id'].isin(component_df['component_id']).all(), 'Component ID not found in component dataframe'

print('Supplier-component dataframe is correct')

In [ ]:
# Supplier dataframe
supplier_df = supplier_component_df[['supplier_id']].drop_duplicates()
supplier_df

In [ ]:
# Create orders dataframe
product_a_factory_1 = 'data/complete_products_factory1_orders_product_A .csv'
product_b_factory_1 = 'data/complete_products_factory1_orders_product_B.csv'
product_c_factory_1 = 'data/complete_products_factory1_orders_product_C.csv'
product_a_factory_2 = 'data/complete_products_factory2_orders_product_A.csv'
product_d_factory_2 = 'data/complete_products_factory2_orders_product_D.csv'

orders_df = None
customers = []

# Process each file
factory_files = [(product_a_factory_1, 'A', 1), (product_b_factory_1, 'B', 1), 
                 (product_c_factory_1, 'C', 1), (product_a_factory_2, 'A', 2),
                 (product_d_factory_2, 'D', 2)]

for file_path, product_id, factory_id in factory_files:
    # Read CSV
    df = pd.read_csv(file_path, delimiter=';')
    
    # Get customer columns (exclude 'Week' column)
    customer_cols = [col for col in df.columns if col != 'Week']
    
    if customer_cols:  # Check if there are customer columns
        # Add new customers to list ----------
        new_customers = [cust.strip() for cust in customer_cols]
        customers.extend([c for c in new_customers if c not in customers])

        # Create a mask for when week resets back to 1
        week_reset_mask = df['Week'].diff() < 0

        # Calculate year changes (cumulative sum of resets)
        year_changes = week_reset_mask.cumsum()

        # Start with year 0, then add the changes
        df['year'] = year_changes
        
        # Pivot the dataframe using week as an identifier ------
        # -> customers will become row values along with their respective amounts
        melted = df.melt(id_vars=['Week', 'year'], value_vars=customer_cols, 
                var_name='customer_id', value_name='amount')
        
        # Clean amount values ------------
        # Make sure amount is numeric
        melted['amount'] = pd.to_numeric(melted['amount'], errors='coerce')
        # Drop empty rows
        melted = melted.dropna(subset=['amount'])
        # Convert to integer
        melted['amount'] = melted['amount'].astype(int)

        # Add product and factory columns
        melted['product_id'] = product_id
        melted['factory_id'] = factory_id
        
        # Combine with main dataframe
        if orders_df is None:
            orders_df = melted
        else:
            orders_df = pd.concat([orders_df, melted])

# Create customers dataframe
customers_df = pd.DataFrame(customers, columns=['customer_id'])

# Create timestamp column by combining year and week columns
orders_df = orders_df.rename(columns={'Week': 'week'})
orders_df['timestamp'] = orders_df['year'].astype(str) + '_' + orders_df['week'].astype(str)

# Reorder columns
orders_df = orders_df[['year', 'week', 'timestamp', 'customer_id', 'factory_id', 'product_id', 'amount']]

# Sort data by year and week and reset index
orders_df = orders_df.sort_values(by=['year', 'week'])
orders_df = orders_df.reset_index(drop=True)

# Replace any 'Unnamed:*' customer_ids with 'Customer_Unknown'
orders_df['customer_id'] = orders_df['customer_id'].replace(r'^Unnamed:.*$', 'Unknown', regex=True)

orders_df

In [ ]:
# Some amount of unnamed customers
orders_df[orders_df['customer_id'].str.startswith('Unknown')]

In [ ]:
# Check that the year and week columns are correct
#print("Unique years in the dataframe: ", orders_df['year'].unique())
#print("Weeks in the second year: ", orders_df[orders_df['year'] == 1]['week'].unique())
#print("Unique year-week combinations: ", orders_df['timestamp'].unique())
assert orders_df['timestamp'].unique().shape[0] == 68, f"Expected 68 unique timestamps, got {orders_df['timestamp'].unique().shape[0]}"

# Check that amount is numeric
assert orders_df['amount'].dtype == 'int64', "Amount column should be of type int64"

# Check for any null values anywhere in the dataframe
assert orders_df.isnull().sum().sum() == 0, "There should be no null values in the dataframe"

# Check that the product_id column is correct
assert orders_df['product_id'].nunique() == 4, "The number of unique products is incorrect"

# Check that all product_ids in orders_df exist in product_df
assert orders_df['product_id'].isin(product_df['product_id']).all(), "Found product_ids in orders_df that don't exist in product_df"

# Check that year and week values are integer
assert orders_df['year'].dtype == 'int64', "Year column should be of type int64"
assert orders_df['week'].dtype == 'int64', "Week column should be of type int64"

print('Orders dataframe is correct')

In [ ]:
# Create factory dataframe
factory_df = pd.DataFrame({'factory_id': orders_df['factory_id'].unique()}).sort_values('factory_id').reset_index(drop=True)
factory_df

In [ ]:
# Create customers dataframe from unique customer_ids in orders_df
customers_df = pd.DataFrame(orders_df['customer_id'].unique(), columns=['customer_id'])
customers_df = customers_df.sort_values('customer_id').reset_index(drop=True)
customers_df

In [ ]:
# Create inventory dataframe
inventory_factory_1 = 'data/Inventory factory 1.csv'
inventory_factory_2 = 'data/Inventory_Factory 2.csv'

# Function to load inventory data
def load_inventory(file_path, factory_id):
    df = pd.read_csv(file_path, delimiter=';')
    # Assume 'Component' is the first column
    component_col = df.columns[0]
    # Find the next non-empty column after 'Component'
    amount_col = None
    for col in df.columns[1:]:
        if df[col].notnull().any():
            amount_col = col
            break
    if amount_col is None:
        raise ValueError(f"No amount column found in {file_path}")
    # Select relevant columns and rename
    df = df[[component_col, amount_col]].dropna()
    df = df.rename(columns={component_col: 'component_id', amount_col: 'amount'})
    df = clean_inventory_data(df)
    df['factory_id'] = factory_id

    return df

def clean_inventory_data(df):
    # Remove any spaces and handle comma separators
    df['amount'] = df['amount'].astype(str).str.replace(' ', '').str.replace(',', '.')
    
    # Fix component_id format
    df['component_id'] = df['component_id'].apply(lambda x: 
        # If already in correct format (A.12), keep as is
        x if len(x) == 4 and '.' in x
        # If dot is missing (A12), insert dot at position 1
        else x[0] + '.' + x[1:] if len(x) == 3
        # If number is single digit (A.1), add leading zero
        else x[0] + '.' + x[2:].zfill(2) if len(x) == 3 and '.' in x
        # For any other case, raise error in check
        else x)

    # Convert amount to integer
    df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
    df['amount'] = df['amount'].astype(int)
    return df

# Load inventory data for both factories
inventory1 = load_inventory(inventory_factory_1, 1)
inventory2 = load_inventory(inventory_factory_2, 2)

# Combine inventories into a single dataframe
inventory_df = pd.concat([inventory1, inventory2], ignore_index=True)

inventory_df

In [ ]:
# Check correct shape
amount_of_rows = inventory_df.shape[0]
assert amount_of_rows == 90, f"Expected 20 rows, got {amount_of_rows}"

# Check for null values in inventory_df
assert inventory_df.isnull().sum().sum() == 0, f"Found null values in inventory_df: {inventory_df.isnull().sum()}"

# Check that each component is found in component_df
assert inventory_df['component_id'].isin(component_df['component_id']).all(), "Found component_ids in inventory_df that don't exist in component_df"

# Check that amount is integer
assert inventory_df['amount'].dtype == 'int64', "Amount column should be of type int64"

# Check that factories are found in factories_df
assert inventory_df['factory_id'].isin(factory_df['factory_id']).all(), "Found factory_ids in inventory_df that don't exist in factory_df"

print('Inventory dataframe is correct')

In [ ]:
# Output path
output_path = 'cleaned_data/'

# Create output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# Save all dataframes to CSV files with semicolon separator
product_df.to_csv(output_path + 'd_product.csv', sep=';', index=False)
component_df.to_csv(output_path + 'd_component.csv', sep=';', index=False)
product_component_dataframe.to_csv(output_path + 'r_product_component.csv', sep=';', index=False)
supplier_component_df.to_csv(output_path + 'r_supplier_component.csv', sep=';', index=False)
supplier_df.to_csv(output_path + 'd_supplier.csv', sep=';', index=False)
orders_df.to_csv(output_path + 'f_orders.csv', sep=';', index=False)
factory_df.to_csv(output_path + 'd_factory.csv', sep=';', index=False)
customers_df.to_csv(output_path + 'd_customers.csv', sep=';', index=False)
inventory_df.to_csv(output_path + 'f_inventory.csv', sep=';', index=False)